In [ ]:
# Auto-reload imports so you don't have to keep restarting the notebook kernel.
%load_ext autoreload
%autoreload complete --print

# Intro to Full Stack: Databases

This applied practice will be focusing mainly on interacting with SQL databases
through the widely used Object-Relational Mapping (ORM) library, SQLAlchemy.
<https://www.reddit.com/r/Python/comments/12xrvwz/comment/jhkgjeb>

TODO: idk why SQLAlchemy decided to switch from their Query API to this new stuff,
the Query API was actually intuitively readable, wtf is a scalar and why do I now
have to explain it? (Btw it seems equivalent to np.item() in numpy)

## Miscellaneous

Run through these after starting/restarting the notebook kernel.

In [18]:
from pathlib import Path

cwd = Path.cwd()

# File paths to the template database and the actual database this workshop will use.
TEMPLATE_DB = cwd / "src" / "template.db"
EXAMPLE_DB = cwd / "my_db.db"

### Enable Printing SQL Statements Made by SQLAlchemy

In [ ]:
import logging

# Log SQL statements made by SQLAlchemy, I suggest you turn this off once you get a hang of it.
LOG_SQLALCHEMY = True

logging.basicConfig()
logging.getLogger("sqlalchemy.engine").setLevel(
    logging.INFO if LOG_SQLALCHEMY else logging.WARNING
)

print(
    "SQLAlchemy logging is enabled."
    if LOG_SQLALCHEMY
    else "SQLAlchemy logging is disabled."
)

## Looking at Schemas

First, view the `User` database schema in [`src/models/user.py`](src/models/user.py).

### Create All Tables

However, declaring the schema is not enough; We have to instruct SQLAlchemy to
`CREATE TABLE` in the database based on the schemas. This has to be done once for
a fresh database.

In [ ]:
from src import Session, Base

# Delete existing database.
try:
    EXAMPLE_DB.unlink(missing_ok=True)
except PermissionError:
    print(
        "The database file is already in use, restart the notebook if really want to delete it."
    )

# Create all tables.
with Session.begin() as sess:
    Base.metadata.create_all(sess.bind)


### Reset Database File

In [ ]:
import shutil

# Overwrite the workshop database with the prepared example database.
shutil.copy(TEMPLATE_DB, EXAMPLE_DB)

## Example: CRUD with Users

In [ ]:
from src.utils import random_name, hash_passwd

EXAMPLE_USERNAME = random_name()
EXAMPLE_PASSWD = hash_passwd("password")
EXAMPLE_USER_ID = None  # Placeholder for the user ID which will be assigned later.
print(f"[Username for Example]\n{EXAMPLE_USERNAME}\n")
print(f"[Password for Example]\n{EXAMPLE_PASSWD}")

### Creating a User

Notice something interesting?

In [ ]:
from src import Session, User

with Session() as sess:
    user = User(username=EXAMPLE_USERNAME, password=EXAMPLE_PASSWD)
    sess.add(user)

    print(f"[`user` Before Commit]\n{user}\n\n", flush=True)
    print("[Committing...]", flush=True)
    sess.commit()
    print("[Committed]", flush=True)

    print("\n\n[Reading from DB...]", flush=True)
    value = str(user)
    print(f"\n\n[`user` After Commit]\n{value}", flush=True)

    EXAMPLE_USER_ID = user.id

Until `sess.commit()` is called, the instance of `User` we created isn't populated
with default values yet. Rather, values like `id` change from None to their actual values only after the commit, even if the field is non-nullable. This means we can't rely on the instance until after the commit.

## Reading a User

For more forms, see <https://docs.sqlalchemy.org/en/20/changelog/migration_20.html#id1>

- TODO: Should I go indepth into Indexes?
- TODO: Double confirm Indexes are implicitly used after `CREATE_INDEX`.
- TODO: Explain why you wouldn't always use an Index.
- TODO: Give analogy for composite indexes (imagine a dict with pairs as the keys that maps to the primary key as the values)

### By ID

In [ ]:
with Session() as sess:
    print(f"[Query by id]\n{EXAMPLE_USER_ID}\n", flush=True)
    user = sess.get(
        User,
        EXAMPLE_USER_ID,
    )
    print(f"\n\n[User retrieved by id]\n{user}")

### By Other `index=True` Columns

In [ ]:
from sqlalchemy import select

with Session() as sess:
    print(f"[Query by username]\n{EXAMPLE_USERNAME}\n", flush=True)
    # `scalar_one()` expects exactly one result, raising `NoResultFound` or `MultipleResultsFound` otherwise.
    sess.execute(select(User).filter_by(username=EXAMPLE_USERNAME)).scalar_one()
    print(f"\n\n[User retrieved by id]\n{user}")

### Searching for Users

### Filtering Users

## Updating a User

### After Being Found Above

### Updating Multiple Users

## Deleting a User

## Your Turn: Transactions CRUD

### Writing the Schema

Fill in the `Transaction` model in [`src/models/transaction.py`](src/models/transaction.py).

### Testing Create

### Testing Read

#### Find by ID

#### Filter by Amount

### Testing Update

### Testing Delete
